In [ ]:
import pyemu
import conda_scripts.load_datasets as lsd
import conda_scripts.sv_budget as sv
import os
import pandas as pd
import geopandas as gpd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
workspace = lsd.model_info.get_mod('son', True)
workspace

In [ ]:
template_ws = os.path.join("temp")

In [ ]:
ml = sv.load_sv_model.get_model(workspace)

In [ ]:
we = pd.read_csv("C:\GSP\sv\model\SV_mod_V2_owhm2\master\FMP_update\sv_owhm_wells_all_on_05142024.in", sep = "\s+")

In [ ]:
we.layer.value_counts()

In [ ]:
data = {'i': [1, 2, 3], 'j': [4, 5, 6]}
data = {'i': [1, ], 'j': [4,]}
df = pd.DataFrame(data)

# List to store new rows
new_rows = []

# Loop through each row in the DataFrame
for _, row in df.iterrows():
    i, j = row['i'], row['j']
    
    # Create 5 new rows for each existing row
    for k in [-1,0,1]:
        for v in [-1,0,1]:
            new_row = {'i': i + k, 'j': j + v}
            new_rows.append(new_row)

# Create a new DataFrame with the new rows
new_df = pd.DataFrame(new_rows)

# Combine the original DataFrame with the new rows
combined_df = pd.concat([df, new_df], ignore_index=True)

In [ ]:
import conda_scripts

In [ ]:
wshed = conda_scripts.load_datasets.load_watersheds.load('son')

In [ ]:
dwr = gpd.read_file(r"T:\arich\GIS\shapefiles\Wells\i07_WellReportStatsBySection\i07_WellCompletionReports\i07_WellCompletionReports.shp")

In [ ]:
dwrc = gpd.clip(dwr.to_crs(2226), wshed)

In [ ]:
dwrc.plot()

In [ ]:
dwrc

In [ ]:
farms

In [ ]:
farms = np.genfromtxt(r"C:\GSP\sv\model\update_2024\scripts\temp\fmp_input\farms\farm2016.dat", delimiter=' ')
farms[(farms==0) | (farms>=81)] = np.nan
plt.imshow(farms)
farms = conda_scripts.arich_functions.array2rc(farms)
farms = farms.loc[farms.value>0]

farms

In [ ]:
lrc = conda_scripts.arich_functions.array2rc(ml.bas6.ibound.array[0])
lrc = lrc.loc[lrc.value>0]

mg = ml.modelgrid
mod = conda_scripts.arich_functions.get_active_mod_df(mg, ml)
lrc = pd.merge(lrc, mod.drop(columns = ['row','col']), on = ['i', 'j'])
lrc = gpd.GeoDataFrame(lrc, geometry = 'geometry', crs = 2226)
lrc = gpd.sjoin(lrc, dwrc)

In [ ]:
lrc.loc[:,['DomWellCou', 'DomWellDep',
       'DomWellD_1', 'DomWellD_2', ]].drop_duplicates()

In [ ]:
lrc.filter(regex = 'PrdWellD')

In [ ]:
lrc = lrc.rename(columns = 
{"PrdWellD_2":'Max Domestic Well Depth',
"PrdWellDep":'Average Domestic Well Depth',
"PrdWellD_1": 'Min Domestic Well Depth'})

In [ ]:
# lrc = lrc.rename(columns = 
# {"DomWellD_2":'Max Domestic Well Depth',
# "DomWellDep":'Average Domestic Well Depth',
# "DomWellD_1": 'Min Domestic Well Depth'})

In [ ]:
lrc = lrc.loc[lrc.loc[:,'Average Domestic Well Depth'].notnull()]

In [ ]:
lrc = conda_scripts.utils.get_pump_layer.wrapper_get_lay(lrc, ml, False)
f = ml.modelgrid.get_lrc([lrc.mfnode])
lrc.loc[:,'layer'] = f[0][0] + 1

In [ ]:
lrc = pd.merge(lrc, farms.rename(columns ={'value':'farm'}).drop(
    columns = ['row', 'column']), on = ['i','j'], how = 'left')

In [ ]:
cols = '#name	farm	layer	row	column	QMAX	date_start	date_end'
cols = cols.split()
cols

In [ ]:
fi = lrc.reindex(columns = cols)
fi.loc[:,'#name'] = 'fmp'+fi.index.astype(str)
fi.loc[:,'QMAX'] = 1e6
fi.loc[:,'date_start'] = 1
fi.loc[:,'date_end'] = 586
fi.head()

In [ ]:
fi = fi.loc[fi.farm.notnull()]

In [ ]:
fi = fi.astype({'farm':int,
          'date_start':int,
          'date_end':int})

In [ ]:
fi.to_csv(r"C:\GSP\sv\model\SV_mod_V2_owhm2\master\FMP_update\allcropsfarms_on_06262024.in",index = False)

In [ ]:
fi.layer.value_counts()